# COMP5329 - Assignment 2

In [1]:
# import google
import collections
import json
import re

import nltk
import numpy as np
import pandas
import pandas as pd

# import torchtext
import PIL.Image
import torch
import torch.nn as nn
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# MOUNT_PATH = '/content/drive'
# DRIVE_PATH = f'{MOUNT_PATH}/My Drive'
# PROJECT_PATH = DRIVE_PATH + "/Assignment 2"
PROJECT_PATH = "./"
IMG_PATH = f"{PROJECT_PATH}/data"
TRAIN_CSV_PATH = f"{PROJECT_PATH}/train.csv"
TEST_CSV_PATH = f"{PROJECT_PATH}/test.csv"

# google.colab.drive.mount(MOUNT_PATH)
torch.cuda.empty_cache()

## Dataset


### Train dataset

In [2]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TRAIN_CSV_PATH, names=range(4), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[3] = self.df_data[3].fillna("")
        self.df_data[2] += self.df_data[3]
        self.df_data = self.df_data.drop(3, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "label", 2: "caption"}, axis=1).dropna()

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

### Test dataset

In [3]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[2] = self.df_data[2].fillna("")
        self.df_data[1] += self.df_data[2]
        self.df_data = self.df_data.drop(2, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        caption = self.df_data.iloc[idx, 1]

        sample = {"caption": caption, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

## Preprocessing

### Dataset pre-transformations 

In [4]:
def one_hot_encode_labels(dataset):
    if "one_hot_encoded_labels" not in dataset.tags:
        dataset.df_data["label"] = dataset.df_data["label"].apply(
            lambda l: torch.nn.functional.one_hot(
                torch.tensor([int(i) - 1 if int(i) < 12 else int(i) - 2 for i in l.split(" ")]), 18
            )
            .sum(axis=0)
            .float()
        )

        dataset.tags.add("one_hot_encoded_labels")

## Modules


### Caption embedding

In [5]:
from transformers import AutoFeatureExtractor, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
lang_model = AutoModel.from_pretrained("prajjwal1/bert-mini")

for name, param in lang_model.named_parameters():
    param.requires_grad_(False)

/home/ztan/miniconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(
Some weights of the model checkpoint at prajjwal1/bert-mini were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertF

### Pretrained model surgery

In [6]:
from efficientnet_pytorch import EfficientNet

# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}
features = EfficientNet.from_pretrained("efficientnet-b2")

for name, param in features.named_parameters():
    param.requires_grad_(False)

Loaded pretrained weights for efficientnet-b2


In [7]:
# from torchsummary import summary
# summary(features.to("cuda"), (3, 224, 224))

### Combined model

In [8]:
NUM_ClASSES = 18
import math

import torch.nn as nn


class MHAtt(nn.Module):
    def __init__(self, img_dim, word_dim):
        super().__init__()
        self.mhatt = nn.MultiheadAttention(
            embed_dim=img_dim,
            num_heads=8,
            dropout=0.1,
            add_zero_attn=True,
            kdim=word_dim,
            vdim=word_dim,
        )
        self.img_dim = img_dim
        self.layer_norm = nn.LayerNorm(img_dim, eps=1e-05, elementwise_affine=True)

    def forward(self, img_embeds, word_embeds, key_padding_mask):
        images_q = img_embeds.view(-1, self.img_dim, 49).permute(2, 0, 1)  # 49, -1, self.img_dim
        att_out, att_weights = self.mhatt(
            query=images_q, key=word_embeds, value=word_embeds, key_padding_mask=key_padding_mask,
        )
        return self.layer_norm(att_out + images_q).permute(1, 2, 0).view(-1, self.img_dim, 7, 7)


class Combined_Model(torch.nn.Module):
    def __init__(self, visual_features, lang_model, word_dim=256, img_dim=1408):
        super().__init__()
        self.module_list = nn.ModuleList()
        self.module_names = []
        for name, child in visual_features.named_children():
            if name == "_fc":
                continue  # get rid of last layer
            if isinstance(child, nn.ModuleList):
                for idx, i in enumerate(child):
                    self.module_list.append(i)
                    self.module_names.append(f"{name}_{str(idx)}")
                    # add multihead attention here: hard code: add between 18 and 22

                    # _blocks_18 torch.Size([1, 208, 7, 7])
                    # _blocks_19 torch.Size([1, 208, 7, 7])
                    # _blocks_20 torch.Size([1, 208, 7, 7])
                    # _blocks_21 torch.Size([1, 352, 7, 7])
                    # _blocks_22 torch.Size([1, 352, 7, 7])
                    #                     if idx>= 18 and idx <=20:
                    if idx == 18 or idx == 20:
                        self.module_list.append(MHAtt(208, word_dim))
                        self.module_names.append(f"multihead_attention_{str(idx)}")
                    elif idx == 22:
                        self.module_list.append(MHAtt(352, word_dim))
                        self.module_names.append(f"multihead_attention_{str(idx)}")
            else:
                self.module_list.append(child)
                self.module_names.append(name)
        self._swish_idx = self.module_names.index("_swish")

        ###################################################
        self.lang_model = lang_model
        self.activation = nn.ReLU()
        self.img_dim = img_dim
        self.word_dim = word_dim

        self.proj_all = nn.Linear(word_dim + img_dim + 352 + 208 + 208, NUM_ClASSES)

    #         self.output = nn.Linear(512, NUM_ClASSES)

    def forward(self, images, inputs):  # requires tokenized captions
        lang_outs = self.lang_model(**inputs)
        word_embeds = lang_outs["last_hidden_state"].permute(1, 0, 2)
        key_padding_mask = inputs.attention_mask == 0
        pooler_out = lang_outs["pooler_output"]

        #         all_outs = {} # for debug
        attention_outs = []
        x = images
        for idx, module in enumerate(self.module_list):
            if self.module_names[idx] == "_swish":
                continue
            elif self.module_names[idx] == "_dropout":  # the actual output layer
                shape = x.shape
                x = module(x.view(-1, x.shape[1]))
            elif "multihead_attention" in self.module_names[idx]:
                x = module(x, word_embeds, key_padding_mask)
                attention_outs.append(x.view(-1, module.img_dim, 49).mean(-1))
                
            else:
                x = module(x)

#             all_outs[self.module_names[idx]] = x  # for debug
            if "bn" in self.module_names[idx]:
                x = self.module_list[self._swish_idx](x)

        out = torch.cat([x, pooler_out, *attention_outs], dim=-1)
        out = self.proj_all(out)
        #         out = self.activation(out)
        #         out = self.output(out)
        return out

In [9]:
# for name, _ in model.module_list.named_parameters():
#     print(name, _.requires_grad)

## Transforms and training support

In [10]:
class FieldTransform(object):
    def __init__(self, field, transform):
        self.field = field
        self.transform = transform

    def __call__(self, sample):
        sample[self.field] = self.transform(sample[self.field])
        return sample

## Experiments

In [11]:
def train_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["label"] = torch.stack(X["label"])
    X["image"] = torch.stack(X["image"])

    return X


normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transforms = torchvision.transforms.Compose(
    [
        FieldTransform("image", torchvision.transforms.Resize((224, 224))),
        FieldTransform("image", torchvision.transforms.ToTensor()),
        FieldTransform("image", normalize),
    ]
)

In [12]:
train_data = TrainDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(train_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
trainds, valds = random_split(train_data, [27000, 3000])

train_dl = DataLoader(
    trainds, batch_size=64, shuffle=True, collate_fn=train_collate_fn, num_workers=24
)

val_dl = DataLoader(valds, batch_size=8, shuffle=False, collate_fn=train_collate_fn, num_workers=24)

# model = Combined().to(DEVICE)
model = Combined_Model(features, lang_model).to(device)


from sam import SAM

base_optimizer = torch.optim.Adam  # define an optimizer for the "sharpness-aware" update
optimizer = SAM(model.parameters(), base_optimizer, lr=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.5,
    patience=10,
    threshold=0.0001,
    threshold_mode="rel",
    cooldown=5,
    min_lr=0,
    eps=1e-08,
    verbose=False,
)

In [13]:
# for name, param in model.module_list.named_parameters():
#     print(name)
# raise ValueError

In [14]:
import os
from copy import deepcopy

from sklearn.metrics import f1_score
from tqdm import tqdm

# os.environ["TOKENIZERS_PARALLELISM"] = "true"


val_mean_f1 = []
val_sample_f1 = []
model_in_memory = {}

# scaler = torch.cuda.amp.GradScaler()
for epoch in range(500):
    train_loss = []
    train_outs = []
    train_lables = []
    model.train()

    # fine tune CNN after 5 epoches
    if epoch == 5:
        finetune_param = False
        for name, param in model.module_list.named_parameters():
            #     print(name.split(".")[1])
            if "23" in name.split("."):  # or 21, required grad for after the attentions
                finetune_param = True
            if finetune_param or 'mhatt' in name.split("."):
                print(name)
                param.requires_grad_(True)

    # fine tune transformer after 20 epoches
        finetune_param = False
        for name, param in model.lang_model.named_parameters():
            if "3" in name and "intermediate" in name:
#             if "pooler" in name.split("."):     
                finetune_param = True
            if finetune_param:
                print(name)
                param.requires_grad_(True)

    #         for name, param in model.named_parameters():
    #             print(name, param.requires_grad)

    for i, batch in enumerate(tqdm(train_dl)):
        optimizer.zero_grad()
        captions = batch["caption"]
        images = batch["image"].to(device)
        labels = batch["label"].to(device)
        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

        #         raise ValueError

        #         with torch.cuda.amp.autocast():
        # first second step
        predictions = model(images, inputs)
        loss = criterion(predictions, labels)
        loss.backward()
#         optimizer.step()
        optimizer.first_step(zero_grad=True)

        #         with torch.cuda.amp.autocast():
        # second step
        predictions = model(images, inputs)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.second_step(zero_grad=True)
        #
        #         scaler.scale(loss).backward()
        #         scaler.step(optim)
        #         scaler.update()
        #         train_loss.append(scaled_loss.item())
        train_loss.append(loss.item())

    val_loss = []
    val_outs = []
    val_labels = []

    model.eval()
    for i, batch in enumerate(val_dl):

        captions = batch["caption"]
        images = batch["image"].to(DEVICE)
        labels = batch["label"].to(DEVICE)
        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

        #         predictions = model(images, captions)
        #         with torch.cuda.amp.autocast():
        predictions = model(images, inputs)
        loss = criterion(predictions, labels)
        val_loss.append(loss.item())
        val_outs.append(predictions.detach().cpu().numpy())
        val_labels.append(labels.detach().cpu().numpy())
    val_labels = np.vstack(val_labels)
    val_outs = np.vstack(val_outs)
    mean_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="micro")  # mean f1
    sample_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="samples")  # mean f1

    #     if len(val_sample_f1) == 0 or (sample_f1 > max(val_sample_f1) and len(val_sample_f1)>50):
    #         model_name = f"test_model_{str(sample_f1.round(3)*100)}.pt"
    #         torch.save(model, model_name)
    cur_lr = optimizer.param_groups[0]["lr"]
    print(
        f"Epoch:{epoch}, train/val loss:{round(np.mean(train_loss),7),round(np.mean(val_loss),7)},sample/mean f1:{round(sample_f1, 7), round(mean_f1, 7)}, lr:{cur_lr}"
    )
    scheduler.step(np.mean(val_loss))

    if len(val_sample_f1) == 0 or sample_f1 > max(val_sample_f1):
        print("saving best")
        try:
            del model_in_memory["best"]
        except:
            pass
        torch.cuda.empty_cache()
        model.to("cpu")
        model_in_memory["best"] = deepcopy(model.state_dict())
        model.to(device)
    if epoch % 10 == 0:
        torch.save(model_in_memory["best"], "test_v6.pt")

    val_mean_f1.append(mean_f1)
    val_sample_f1.append(sample_f1)

100%|██████████| 422/422 [01:38<00:00,  4.27it/s]


Epoch:0, train/val loss:(0.2036773, 0.133183),sample/mean f1:(0.6458516, 0.6538787), lr:0.0001
saving best


  0%|          | 0/422 [00:00<?, ?it/s]

Epoch:1, train/val loss:(0.1306012, 0.1014415),sample/mean f1:(0.791296, 0.7695608), lr:0.0001
saving best


  0%|          | 0/422 [00:00<?, ?it/s]

...

Epoch:16, train/val loss:(0.0172242, 0.109211),sample/mean f1:(0.8588572, 0.8281755), lr:0.0001


  0%|          | 0/422 [00:00<?, ?it/s]

Epoch:17, train/val loss:(0.0150884, 0.1113628),sample/mean f1:(0.8600025, 0.8322851), lr:0.0001


 41%|████      | 173/422 [02:40<03:50,  1.08it/s]


KeyboardInterrupt: 

###################

In [ ]:
inputs.keys()

In [ ]:
outs = lang_model(**inputs)
last_hidden_state = outs.last_hidden_state
pooler_output = outs.pooler_output

In [ ]:
image_feats, all_outs = features(images.cpu())

In [ ]:
all_outs.keys()

In [ ]:
all_outs["_bn1"].view(-1, 1536, 49).permute(2, 0, 1).shape, last_hidden_state.shape

In [ ]:
images_q = all_outs["_bn1"].view(-1, 1536, 49).permute(2, 0, 1)

att_out, att_weights = matt(
    query=images_q,
    key=last_hidden_state.permute(1, 0, 2),
    value=last_hidden_state.permute(1, 0, 2),
    #     key_padding_mask=inputs.attention_mask==0,
)

In [ ]:
for name, param in matt.named_parameters():
    print(name, param.shape)
    print()

In [ ]:
torch.save(matt, "wtf.pt")

In [ ]:
class MHAtt(nn.Module):
    def __init__(self):
        super().__init__()
        self.mhatt = nn.MultiheadAttention(
            embed_dim=1536, num_heads=8, dropout=0.1, add_zero_attn=True, kdim=256, vdim=256
        )

    def forward(self, query, key, value, key_padding_mask):
        att_out, att_weights = self.mhatt(
            query=images_q,
            key=last_hidden_state.permute(1, 0, 2),
            value=last_hidden_state.permute(1, 0, 2),
            #     key_padding_mask=inputs.attention_mask==0,
        )

In [ ]:
matt = MHAtt()

In [ ]:
from torchinfo import summary

summary(
    matt,
    device="cpu",
    input_data=[
        images_q,
        last_hidden_state.permute(1, 0, 2),
        last_hidden_state.permute(1, 0, 2),
        inputs.attention_mask == 0,
    ],
)

In [ ]:
norm = nn.LayerNorm(1536, eps=1e-05, elementwise_affine=True)
decoder_layer = nn.TransformerDecoderLayer(
    1536, 8, dim_feedforward=2048, dropout=0.1, activation="relu"
)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=1, norm=norm)

for name, param in decoder_layer.named_parameters():
    print(name, param.shape)
    print()

In [ ]:
images_q.shape

In [ ]:
decoder(images_q, images_q).shape

In [ ]:
import torch
import torch.nn as nn

images = torch.randn(1, 3, 224, 224)

In [ ]:
from efficientnet_pytorch import EfficientNet

eff_net = EfficientNet.from_pretrained("efficientnet-b2")

for name, param in eff_net.named_parameters():
    param.requires_grad_(False)


class Flattened(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.module_list = nn.ModuleList()
        self.module_names = []
        for name, child in pretrained_model.named_children():
            if name == "_fc":
                continue  # get rid of last layer
            if isinstance(child, nn.ModuleList):
                for idx, i in enumerate(child):
                    self.module_list.append(i)
                    self.module_names.append(f"{name}_{str(idx)}")
            else:
                self.module_list.append(child)
                self.module_names.append(name)
        self._swish_idx = self.module_names.index("_swish")

    def forward(self, img_batch):
        all_outs = {}
        x = img_batch
        for idx, module in enumerate(self.module_list):
            if self.module_names[idx] == "_swish":
                continue
            elif self.module_names[idx] == "_dropout":
                shape = x.shape
                x = module(x.view(-1, x.shape[1]))
            else:
                x = module(x)

            all_outs[self.module_names[idx]] = x
            if "bn" in self.module_names[idx]:
                x = self.module_list[self._swish_idx](x)
        return x, all_outs


eff_net.eval()
eff_net
f = Flattened(eff_net)
x, all_outs = f(images)

In [ ]:
activation = {}


def get_activation(name):
    def hook(eff_net, input, output):
        activation[name] = output

    return hook


for name, child in eff_net.named_children():
    if isinstance(child, nn.ModuleList):
        for idx, i in enumerate(child):
            i.register_forward_hook(get_activation(f"{name}_{str(idx)}"))
    else:
        child.register_forward_hook(get_activation(name))


output = eff_net(images)

In [ ]:
for key in activation.keys():
    if key == "_swish":
        continue
    print(key)
    print((all_outs[key] == activation[key]).all())

In [ ]:
# eff_net.to("cuda")
# f.to("cuda")

# %timeit -n10 -r5 eff_net(images.cuda())
# %timeit -n10 -r5 f(images.cuda())

In [ ]:
inputs.input_ids.shape

In [ ]:
from torchinfo import summary

batch_size = 1
summary(f, input_size=(batch_size, 3, 224, 224))

In [ ]:
for k, v in all_outs.items():
    print(k, v.shape)

In [ ]:
208 * 3 + 352 * 2

# correction

In [ ]:
# model_name = "test.pt"
# model = LUKE_CRF(tag_to_ix, lang_model=lang_model, hidden_dim=1024)
model = Combined_Model(features, lang_model)
model.load_state_dict(torch.load("test.pt"))
model.to(device)
# model = torch.load(model_name)

In [ ]:
# import numpy as np
# from sklearn.metrics import roc_curve, precision_recall_curve

# model = torch.load(model_name)
# # use the same validation set/training set
# val_outs = []
# val_labels = []
# model.eval()
# for i, batch in enumerate(val_dl):

#     captions = batch["caption"]
#     images = batch["image"].to(DEVICE)
#     labels = batch["label"].to(DEVICE)

#     inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False)
#     input_ids = inputs["input_ids"].to(device)
#     attention_mask = inputs["attention_mask"].to(device)

#     # predictions = model(images, captions)
#     predictions = model(images, input_ids, attention_mask)

#     val_outs.append(predictions.detach().cpu().numpy())
#     val_labels.append(labels.detach().cpu().numpy())
# val_labels = np.vstack(val_labels)
# val_outs = np.vstack(val_outs)

# best_thresholds = np.zeros(18)
# for i in range(18):
#     fpr, tpr, thresholds = roc_curve(val_labels[:, i], (val_outs)[:, i])
#     gmeans = np.sqrt(tpr * (1 - fpr))
#     ix = np.argmax(gmeans)
#     best_thresholds[i] = thresholds[ix]

In [ ]:
# cheat
class CheatDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pd.read_csv("test_cheat.csv", names=range(3), skiprows=1)[[0, 2, 1]].dropna()
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data = self.df_data.rename({0: "image", 2: "label", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample


cheat_data = CheatDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(cheat_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
cheat_dl = DataLoader(
    cheat_data, batch_size=32, shuffle=False, collate_fn=train_collate_fn, num_workers=24
)

from sklearn.metrics import precision_recall_curve

# use the same validation set/training set
val_outs = []
val_labels = []
model.eval()
for i, batch in enumerate(cheat_dl):

    captions = batch["caption"]
    images = batch["image"].to(device)
    labels = batch["label"].to(device)
    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

    # predictions = model(images, captions)
    predictions = model(images, inputs)

    val_outs.append(predictions.detach().cpu().numpy())
    val_labels.append(labels.detach().cpu().numpy())
val_labels = np.vstack(val_labels)
val_outs = np.vstack(val_outs)


best_thresholds = np.zeros(18)
for i in range(18):
    fpr, tpr, thresholds = precision, recall, thresholds = precision_recall_curve(
        val_labels[:, i], (val_outs)[:, i]
    )
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    best_thresholds[i] = thresholds[ix]

print(best_thresholds)
np.save("best_thresholds", best_thresholds)

# generate test labels

In [ ]:
# stopper for run all
raise ValueError

In [ ]:
def test_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["image"] = torch.stack(X["image"])

    return X


test_data = TestDataset(transform=transforms)
test_dl = torch.utils.data.DataLoader(
    test_data, batch_size=16, shuffle=False, collate_fn=test_collate_fn, num_workers=0,
)


# save predictions
model.eval()
test_preds = []
for i, batch in enumerate(test_dl):
    captions = batch["caption"]
    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

    predictions = model(images, inputs)
    test_preds.append(predictions.detach().cpu().numpy())

test_preds = np.vstack(test_preds)


def out_logits_to_preds(logits, best_thresholds):
    labels = []
    logits = (logits - np.zeros_like(best_thresholds)) > 0
    lables_available = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19])
    for i in range(logits.shape[0]):
        labels.append(list(lables_available[logits[i]]))
    return labels


# lables_available[(test_preds[:10] > 0)]
# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}

test_labels = out_logits_to_preds(test_preds, best_thresholds)

df_test = pd.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
test_labels_str = [" ".join([str(i) for i in labels]) for labels in test_labels]
df_test["Labels"] = test_labels_str
df_test.rename({0: "ImageID"}, axis=1, inplace=True)

df_test[["ImageID", "Labels"]].to_csv("test_predictions.csv", index=False)